## AutoML

In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd


# Create TabularDataset using TabularDatasetFactory

#  path to URL 
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

In [14]:
ds = TabularDatasetFactory.from_delimited_files(url)

'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

In [18]:
#  read remote URL data to DataFrame
ds = TabularDatasetFactory.from_delimited_files(url,header = False)

In [19]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#automl settings have the optios to assign the dataframe and identify a target variable within it.
datafinal = pd.concat([x,y], axis = 1)
datafinal.head()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

UserErrorException: UserErrorException:
	Message: Invalid argument type for `path`. It can be single value or a list of: string starting with "http://" or "https://", `azureml.data.datapath.DataPath` object, or tuple of `azureml.core.Datastore` object and string for relative path in the datastore.Url of blob can't contains SAS token, please use datastore instead.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Invalid argument type for `path`. It can be single value or a list of: string starting with \"http://\" or \"https://\", `azureml.data.datapath.DataPath` object, or tuple of `azureml.core.Datastore` object and string for relative path in the datastore.Url of blob can't contains SAS token, please use datastore instead."
    }
}

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    compute_target = compute_target,
    primary_metric = 'accuracy',
    n_cross_validations = 3, #typically 5, used to make time go quicker.
    task = 'classification',
    training_data = ds,
    label_column_name = 'y',
    enable_onnx_compatible_models = True)

In [16]:
# Submit your automl run
experiment = Experiment(ws, 'udacity_automl')
automl_run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster-01 with default configuration
Running on remote compute: cpu-cluster-01
Parent Run ID: AutoML_573ad1d1-40a4-457f-a797-cdccc223a339

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the

In [19]:
pip install azureml-train-automl-runtime

     |████████████████████████████████| 119 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 219 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.1 MB/s  eta 0:00:01
     |████████████████████████████████| 371 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 186 kB 102.0 MB/s eta 0:00:01
     |████████████████████████████████| 107 kB 98.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 59.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 93.0 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 7.4 MB 85.7 MB/s eta 0:00:01
     |████████████████████████████████| 28.2 MB 30.5 MB/s eta 0:00:01


     |████████████████████████████████| 114.1 MB 67 kB/s s eta 0:00:01
     |████████████████████████████████| 53 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 5.9 MB 103.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.9 MB 52.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 54.2 MB/s eta 0:00:01


ERROR: azureml-widgets 1.24.0 has requirement azureml-core~=1.24.0, but you'll have azureml-core 1.18.0.post4 which is incompatible.
ERROR: azureml-widgets 1.24.0 has requirement azureml-telemetry~=1.24.0, but you'll have azureml-telemetry 1.18.0.post1 which is incompatible.
ERROR: azureml-train-core 1.24.0 has requirement azureml-core~=1.24.0, but you'll have azureml-core 1.18.0.post4 which is incompatible.
ERROR: azureml-train-core 1.24.0 has requirement azureml-telemetry~=1.24.0, but you'll have azureml-telemetry 1.18.0.post1 which is incompatible.
ERROR: azureml-tensorboard 1.24.0 has requirement azureml-core~=1.24.0, but you'll have azureml-core 1.18.0.post4 which is incompatible.
ERROR: azureml-sdk 1.24.0 has requirement azureml-core~=1.24.0, but you'll have azureml-core 1.18.0.post4 which is incompatible.
ERROR: azureml-sdk 1.24.0 has requirement azureml-dataset-runtime[fuse]~=1.24.0, but you'll have azureml-dataset-runtime 1.18.0 which is incompatible.
ERROR: azureml-sdk 1.24.0

  Attempting uninstall: azureml-dataprep-rslex
    Found existing installation: azureml-dataprep-rslex 1.9.0
    Uninstalling azureml-dataprep-rslex-1.9.0:
      Successfully uninstalled azureml-dataprep-rslex-1.9.0
  Attempting uninstall: azureml-dataprep-native
    Found existing installation: azureml-dataprep-native 30.0.0
    Uninstalling azureml-dataprep-native-30.0.0:
      Successfully uninstalled azureml-dataprep-native-30.0.0
  Attempting uninstall: azureml-dataprep
    Found existing installation: azureml-dataprep 2.11.1
    Uninstalling azureml-dataprep-2.11.1:
      Successfully uninstalled azureml-dataprep-2.11.1
  Attempting uninstall: azureml-dataset-runtime
    Found existing installation: azureml-dataset-runtime 1.24.0
    Uninstalling azureml-dataset-runtime-1.24.0:
      Successfully uninstalled azureml-dataset-runtime-1.24.0
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 20.0.1
    Uninstalling pyOpenSSL-20.0.1:
      Successfully unins

In [20]:
import azureml.train.automl

import azureml.automl.core
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_automl_run, best_automl_onnx_model = automl_run.get_output(return_onnx_model = True)
OnnxConverter.save_onnx_model(best_automl_onnx_model, file_path = 'outputs/best_automl_model.onnx')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.